In [ ]:
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
#connection
DATABASE = 'YOUR_DATABASE'
schema = 'YOUR_SCHEMA'
username = 'YOUR_USERNAME' #update for your own account
account = 'YOUR_ACCOUNT.YOUR_REGION'

kwargs = {"authenticator": "YOUR_AUTHENTICATOR_METHOD"} #options

connection = snowflake.connector.connect(
    user       = username,
    password   = "YOUR_PASSWORD",
    account    = account,
    **kwargs
    )

#specify schema
cur = connection.cursor()

# engine
engine = create_engine(URL(
    account = 'YOUR_ACCOUNT.YOUR_REGION',
    user = 'YOUR_USERNAME',
    database = 'YOUR_DB_NAME',
    schema = 'YOUR_SCHEMA',
    authenticator='YOUR_AUTHENTICATOR_METHOD'))          


In [ ]:
# Date lists
var_list = ['DATE_1', 'DATE_2', 'DATE_3', 'DATE_4', 'DATE_5', 'DATE_6', 'DATE_7', 'DATE_8', 'DATE_9', 'DATE_10', 'DATE_11']
mth_list = ['MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5', 'MONTH_6', 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10', 'MONTH_11']
cur_list = ['START_DATE_1', 'START_DATE_2', 'START_DATE_3', 'START_DATE_4', 'START_DATE_5', 'START_DATE_6', 'START_DATE_7', 'START_DATE_8', 'START_DATE_9', 'START_DATE_10', 'START_DATE_11']
cur_list2= ['END_DATE_1', 'END_DATE_2', 'END_DATE_3', 'END_DATE_4', 'END_DATE_5', 'END_DATE_6', 'END_DATE_7', 'END_DATE_8', 'END_DATE_9', 'END_DATE_10', 'END_DATE_11']

# SQL query template
query = """
select pt.COLUMN_1
         , pt.COLUMN_2
         , {var2}.COLUMN_3
         , {var2}.COLUMN_4
         ...
         , {var2}.COLUMN_N
from YOUR_TABLE_1 pt
join YOUR_TABLE_2_{var1} {var2}
on pt.ID_COLUMN={var2}.ID_COLUMN
where pt.DATE_COLUMN between '{var3}' and '{var4}'
and pt.COLUMN_FILTER_1 = 'FILTER_VALUE'
and pt.COLUMN_FILTER_2 in ('FILTER_VALUE_1') and COLUMN_FILTER_3 = 1
"""

dataframes = []

for var1, var2, var3, var4 in zip(var_list, mth_list, cur_list, cur_list2):
    # Format and execute the modified query
    query_to_execute = query.format(var1=var1, var2=var2, var3=var3, var4=var4)
    df = pd.read_sql(query_to_execute, connection)
    df.columns = df.columns.str.lower() 
    dataframes.append(df)

final_dataframe = pd.concat(dataframes, ignore_index=True)
final_dataframe.drop_duplicates(subset='YOUR_ID_COLUMN', keep='last', inplace=True)


In [ ]:
final_dataframe

In [ ]:
# Write the DataFrame to Snowflake
table_name = "loop_example"
final_dataframe.to_sql(table_name, engine, if_exists='replace', index=False)  # 'replace' will replace the table if it exists, 'append' will insert new rows